<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/gsi073_aula0_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

Neste notebook, um tipo de Support Vector Machine Linear.


## Preparação dos dados

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

# Preparar o dataset
iris = datasets.load_iris()
X = iris.data; y = iris.target

X = X[y != 1] ; y = y[y != 1] # versicolor
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1  # SVM espera rótulos em {-1, +1}

X = torch.tensor(X, dtype=torch.float32) # Tensor é um tipo especial que suporta muitas dimensões

A nossa Support Vector Machine é basicamente um hiperplano definido por *w* e *b* que melhor separa as classes.

Boa! 👏 Você acabou de montar a parte mais chatinha da preparação de dados para fazer uma **SVM “estilo SVM”** em PyTorch: binarizou o problema do Iris e colocou os rótulos em **{-1, +1}**, que é exatamente o que a formulação clássica de SVM usa.

Vamos analisar o que o seu código fez e já te mostro o próximo passo (definir o modelo e a loss tipo hinge).

---

## 🧠 O que o seu código fez

```python
iris = datasets.load_iris()
X = iris.data
y = iris.target
```

* Carregou o Iris (150 amostras, 3 classes: 0=setosa, 1=versicolor, 2=virginica)

```python
X = X[y != 1]
y = y[y != 1]
```

* Aqui você **removeu a classe 1 (versicolor)**.
* Então sobraram **duas classes**:

  * `0` → setosa
  * `2` → virginica
* Isso transformou o problema em **binário**, o que é ótimo para testar uma SVM “na mão”.

```python
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1
```

* Convertendo os rótulos para tensor…
* …e trocando a classe `0` por `-1`.
* Resultado final:

  * setosa → **-1**
  * virginica → **2** ❗

⚠️ Aqui tem um detalhe importante:
você **não converteu a classe 2 para +1**. Então seus rótulos ficaram assim:

* `-1` para a classe 0
* `2` para a classe 2

Mas para SVM o ideal é **{-1, +1}**. Então precisamos fazer mais uma linha para mapear o 2 → +1.

---

## ✅ Versão corrigida da preparação

```python
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

# 1. Carregar dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 2. Ficar só com 2 classes (excluir a classe 1 = versicolor)
mask = (y != 1)
X = X[mask]
y = y[mask]   # agora y tem só 0 e 2

# 3. Converter para {-1, +1}
#  - classe 0 (setosa)  -> -1
#  - classe 2 (virginica) -> +1
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1.0
y[y == 2] = 1.0

# 4. Converter X para tensor
X = torch.tensor(X, dtype=torch.float32)

print("Shape X:", X.shape)
print("Valores únicos de y:", y.unique())
```

Se você rodar isso, deve ver algo assim:

```text
Shape X: torch.Size([100, 4])
Valores únicos de y: tensor([-1.,  1.])
```

👆 Isso significa:

* Ficamos com 100 amostras (50 setosa, 50 virginica)
* Agora o problema está no formato perfeito para uma SVM.

---

## 🧭 Próximo passo

Agora que os dados estão prontos, dá pra fazer uma SVM “na unha” em PyTorch de 2 jeitos:

1. **SVM primal simplificada**

   * modelo linear: `f(x) = w·x + b`
   * loss = hinge + L2
   * otimiza com SGD/Adam

2. **“Fake SVM” com BCE**

   * manter estrutura de SVM mas usar loss de rede neural

Como você mencionou **“termo de regularização” e “perda de classificação”**, acho que você quer ver a **versão com hinge loss mesmo**. Ela fica assim 👇

```python
class LinearSVM(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(n_features, 1))
        self.b = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # retorna o score (não é probabilidade)
        return x @ self.w + self.b  # shape: [N, 1]


def hinge_loss(outputs, y, C=1.0):
    # y: [N] com valores em {-1, +1}
    # outputs: [N, 1]
    # hinge: max(0, 1 - y * f(x))
    y = y.view(-1, 1)
    classification_loss = torch.clamp(1 - y * outputs, min=0).mean()
    # termo de regularização (1/2 ||w||^2)
    return 0.5 * (model.w ** 2).sum() + C * classification_loss
```

Se quiser, já te monto o loop de treino completo para essa SVM em PyTorch com print de loss. Quer que eu faça na sequência?


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Carregar e preparar dados (Iris, 2 classes)
iris = datasets.load_iris()
X = iris.data
y = iris.target

# manter só setosa (0) e virginica (2)
mask = (y != 1)
X = X[mask]
y = y[mask]         # agora só 0 e 2

# converter rótulos para {-1, +1}
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1.0
y[y == 2] = 1.0

# 2. train/test split + normalização (muito importante p/ SVM)
X_train, X_test, y_train, y_test = train_test_split(
    X, y.numpy(), test_size=0.2, random_state=42, shuffle=True, stratify=y.numpy()
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# converter para tensores
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32)

# 3. Definir modelo SVM linear
class LinearSVM(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        # w: [n_features, 1], b: [1]
        self.w = nn.Parameter(torch.zeros(n_features, 1))
        self.b = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # x: [N, d]
        # return: [N, 1] — score (não é probabilidade)
        return x @ self.w + self.b

model = LinearSVM(n_features=4)

# 4. Definir hinge loss + regularização
def svm_loss(model, outputs, y, C=1.0):
    # outputs: [N, 1], y: [N]
    y = y.view(-1, 1)  # [N, 1]
    # hinge: max(0, 1 - y * f(x))
    hinge = torch.clamp(1 - y * outputs, min=0)  # [N, 1]
    classification_loss = hinge.mean()
    # regularização L2: 1/2 ||w||^2
    reg = 0.5 * torch.sum(model.w ** 2)
    # total
    return reg + C * classification_loss, classification_loss, reg

# 5. Otimizador
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 6. Treino
EPOCHS = 1000
C = 1.0  # quanto maior, mais o modelo tenta acertar o treino (menos margem)
for epoch in range(1, EPOCHS + 1):
    optimizer.zero_grad()
    outputs = model(X_train_t)   # [N, 1]
    loss, cls_loss, reg_loss = svm_loss(model, outputs, y_train_t, C=C)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0 or epoch == 1:
        # acurácia de treino
        with torch.no_grad():
            preds_train = torch.sign(model(X_train_t)).view(-1)  # -1 ou +1
            acc_train = (preds_train == y_train_t).float().mean().item()
        print(f"[{epoch:4d}/{EPOCHS}] loss={loss.item():.4f}  cls={cls_loss.item():.4f}  reg={reg_loss.item():.4f}  acc_train={acc_train:.3f}")

# 7. Avaliação no TESTE
with torch.no_grad():
    scores_test = model(X_test_t).view(-1)
    preds_test = torch.sign(scores_test)  # -1 ou +1
    acc_test = (preds_test == y_test_t).float().mean().item()

print("\n=== Avaliação no TESTE ===")
print(f"Acurácia: {acc_test:.3f}")

# matriz de confusão (convertendo p/ {0,1} só pra visualizar melhor)
y_test_bin = (y_test_t == 1).int().numpy()
preds_test_bin = (preds_test == 1).int().numpy()
cm = confusion_matrix(y_test_bin, preds_test_bin)
print("Matriz de confusão:\n", cm)


Isso ficou lindo 😎

Vamos dissecar o que o seu output está dizendo, porque ele está mostrando uma **SVM linear funcionando exatamente como no papel**.

---

## 1. O treino

```text
[   1/1000] loss=1.0000  cls=1.0000  reg=0.0000  acc_train=1.000
[ 100/1000] loss=0.2398  cls=0.0585  reg=0.1813  acc_train=1.000
...
[1000/1000] loss=0.2384  cls=0.0508  reg=0.1876  acc_train=1.000
```

O que isso conta:

* **na época 1**:

  * `cls=1.0000`: vários pontos ainda estavam “dentro” da margem (ou no limite) → hinge loss alta.
  * `reg=0.0000`: os pesos começaram em zero.
  * `acc_train=1.000`: mesmo com hinge alta, **todos já estavam do lado certo do hiperplano**. Isso acontece porque o Iris (só setosa vs virginica) é **muito bem separável**.

* **a partir da época 100**:

  * `cls` (perda de classificação) caiu para ~0.05 → ou seja, quase todo mundo ficou **fora da margem**.
  * `reg` subiu para ~0.187 → o modelo “esticou” um pouco o vetor (w) para **aumentar a margem**, e a regularização começou a pesar.
  * `loss` total estabilizou em **0.2384** → isso é o equilíbrio clássico da SVM:
    [
    \text{loss total} = \underbrace{0.5|w|^2}*{\text{regularização}} + C \cdot \underbrace{\text{hinge}}*{\text{classificação}}
    ]

📌 O que isso mostra?
Que o modelo **chegou muito rápido a 100% de acerto** e depois ficou **apenas ajustando a margem** — exatamente o comportamento esperado de uma SVM quando os dados são **linearmente separáveis**.

---

## 2. Treino com acurácia 1.000 desde o começo

Isso pode parecer estranho (“ué, já acertou tudo na época 1?”), mas faz sentido:

* você usou **só duas classes bem separadas do Iris** (setosa e virginica);
* você **normalizou** os dados;
* o modelo é **linear**, e o problema é **linearmente separável**.

Então, o que a SVM faz depois disso não é mais “aprender a classificar”, e sim:

> “ok, já separei — agora vou achar **a melhor separação possível** (a de maior margem)”.

É por isso que:

* `acc_train` ficou 1.000 o tempo todo ✅
* mas `cls` e `reg` ficaram se ajustando até estabilizar

Ou seja:
**classificação perfeita** + **busca da margem ótima**.

---

## 3. Avaliação no teste

```text
=== Avaliação no TESTE ===
Acurácia: 1.000
Matriz de confusão:
 [[10  0]
 [ 0 10]]
```

Isso aqui é o cenário ideal:

* 10 negativos (setosa) → todos corretos
* 10 positivos (virginica) → todos corretos
* nenhuma confusão, nenhum falso positivo, nenhum falso negativo

👉 **Sua SVM generalizou perfeitamente.**

Isso confirma duas coisas importantes:

1. **O dataset nesse recorte (0 vs 2) é mesmo linearmente separável.**
2. **A implementação da SVM em PyTorch está correta.**
   (se tivesse algum erro de sinal, shape, ou rótulo {0,1} vs {-1,+1}, você veria 1 ou 2 erros aqui)

---

## 4. Papel dos dois termos (ficou visível!)

Olha que legal:

* `cls ≈ 0.05`
  → quase ninguém mais está violando a margem
  → **perda de classificação baixa**

* `reg ≈ 0.187`
  → o modelo está pagando um custo por ter um (w) com certo tamanho
  → **isso é o termo de regularização empurrando a solução para uma margem máxima**

Ou seja: você viu **ao vivo** o trade-off que a gente comentou:

[
\text{Loss total} = \underbrace{0.5|w|^2}*{\text{regularização}} + C \cdot \underbrace{\text{hinge}}*{\text{classificação}}
]

Se você agora **aumentar o C** (ex.: `C = 10` no `svm_loss`), o que vai acontecer?

* o modelo vai tentar zerar ainda mais o `cls`
* mas o `reg` vai subir → margem fica menor
* em dados com ruído, isso levaria a overfitting

Se você **diminuir o C** (ex.: `C = 0.1`):

* o modelo aceita um pouco mais de violação de margem (`cls` sobe)
* mas o peso dá uma encolhida (`reg` desce)
* você ganha **margem maior** → mais robusto

Dá pra testar isso direto no seu código trocando só o `C` 👍

---

## 5. Então… esse é um bom modelo preditivo?

**Sim.** Para esse recorte do problema (Iris 0 vs 2), ele está **perfeito**.

Mas é importante dizer:

* você escolheu **um problema fácil** (linearmente separável);
* em problemas reais, com ruído, 3+ classes, ou dados não lineares, a SVM **não vai ficar 1.000 / 1.000 assim**;
* aí entra: kernel, modelo neural ou features melhores.

---

## 6. O que isso te mostra na aula de LLMs

Isso que você acabou de fazer mostra **a ponte**:

* SVM → separa vetores por produto interno + regularização
* LLM → também compara vetores por produto interno (Q·Kᵀ), mas **aprende o espaço de vetores**
* GPU → máquina de produto interno

Você acabou de ver o caso “puro” do aprendizado estatístico acontecendo.
Agora dá pra mostrar em aula: “olha, a mesma ideia de separar no espaço vetorial é o que o Transformer faz, mas com atenção e embeddings”.

---

Se quiser, o próximo passo natural é:

1. repetir o mesmo código **sem normalizar** e ver que a loss não fica tão bonitinha;
2. ou trocar pra **kernel RBF** no sklearn e comparar;
3. ou remontar o mesmo problema mas **com as 3 classes** e mostrar por que a SVM precisa do truque “one-vs-rest”.

Quer ir pra qual desses? 👀



In [ ]:
# Definir parâmetros treináveis da Support Vector Machine: w e b
n_features = X.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# === Hiperparâmetros ===
learning_rate = 0.01
epochs = 300
optimizer = optim.Adam([w, b], lr=learning_rate)

Perfeito 👏 — esse trecho que você acabou de colocar está montando a versão “manual” dos parâmetros da SVM, e é uma ótima forma de entender como ela funciona **por dentro**.

Vamos destrinchar o que você fez e o que viria a seguir 👇

---

## 🧠 O que o código faz

```python
n_features = X.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
```

### 🔹 `w`

* É o **vetor de pesos** da SVM (a normal do hiperplano).
* Tem dimensão `(n_features, 1)` — ou seja, um peso para cada feature do Iris.
* Inicializado aleatoriamente com distribuição normal.

### 🔹 `b`

* É o **bias** (termo de deslocamento).
* Inicializado como zero.
* Também tem gradiente habilitado → vai ser ajustado durante o treinamento.

---

## ⚙️ Hiperparâmetros

```python
learning_rate = 0.01
epochs = 300
optimizer = optim.Adam([w, b], lr=learning_rate)
```

### 🔹 `learning_rate`

* Controla o passo de atualização dos parâmetros.
* O valor 0.01 é razoável para Adam.

### 🔹 `epochs`

* Número de vezes que o modelo vai ver o conjunto de treino inteiro.

### 🔹 `optimizer`

* Adam vai atualizar diretamente `w` e `b`.
* Isso é legal porque **você não está usando um `nn.Module`**, então precisa passar os parâmetros manualmente.

---

## 🧩 Próximo passo — definir a **função de perda da SVM**

Agora você precisa criar a **hinge loss com regularização**, igual à que usamos antes com a classe `LinearSVM`:

```python
def svm_loss(X, y, w, b, C=1.0):
    # Forward: f(x) = Xw + b
    outputs = X @ w + b

    # Hinge loss: max(0, 1 - y * f(x))
    hinge = torch.clamp(1 - y.view(-1, 1) * outputs, min=0)
    classification_loss = hinge.mean()

    # Regularização L2: 1/2 ||w||^2
    reg = 0.5 * torch.sum(w ** 2)

    return reg + C * classification_loss, classification_loss, reg
```

---

## 🔁 Loop de treinamento

Você pode usar o mesmo padrão de loop:

```python
C = 1.0
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()

    total_loss, cls_loss, reg_loss = svm_loss(X, y, w, b, C=C)
    total_loss.backward()
    optimizer.step()

    if epoch % 50 == 0 or epoch == 1:
        print(f"[{epoch:4d}/{epochs}] loss={total_loss.item():.4f}  cls={cls_loss.item():.4f}  reg={reg_loss.item():.4f}")
```

---

## 🧮 Avaliação final

Após o treino:

```python
with torch.no_grad():
    preds = torch.sign(X @ w + b)
    acc = (preds.view(-1) == y).float().mean()
    print(f"Acurácia final: {acc:.3f}")
```

---

## 🧭 Comparando com o modelo anterior

Esse código é equivalente ao que você fez antes com `nn.Module`, mas aqui:

* tudo é explícito (você vê exatamente os tensores `w` e `b`);
* o otimizador atualiza diretamente esses tensores;
* você entende claramente onde está cada termo:
  regularização (`0.5 * ||w||²`) e hinge (`max(0, 1 - y f(x))`).

Se você rodar, deve ver a **mesma convergência rápida** (loss estabilizando ~0.23 e acurácia = 1.000).

---

Quer que eu adicione **a parte de divisão treino/teste e normalização** dentro desse código para deixar ele completinho e comparável com o modelo anterior?


In [ ]:
import torch
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

# 1) Carregar dataset
iris = datasets.load_iris()
X = iris.data      # shape (150, 4)
y = iris.target    # 0=setosa, 1=versicolor, 2=virginica

# 2) Ficar só com 2 classes: 0 e 2 (tirar 1)
mask = (y != 1)
X = X[mask]
y = y[mask]   # agora tem só 0 e 2

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)

# 4) Normalizar (fit no treino, transform no teste)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 5) Converter para tensores e trocar rótulos para {-1, +1}
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32)

# SVM usa -1 e +1
y_train_t[y_train_t == 0] = -1.0
y_train_t[y_train_t == 2] =  1.0
y_test_t[y_test_t == 0]   = -1.0
y_test_t[y_test_t == 2]   =  1.0

# 6) Definir parâmetros treináveis (w e b) "na mão"
n_features = X_train_t.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)  # pesos
b = torch.zeros(1, requires_grad=True)              # bias

# 7) Hiperparâmetros
learning_rate = 0.01
epochs = 300
C = 1.0   # peso da perda de classificação (como o C da SVM)
optimizer = optim.Adam([w, b], lr=learning_rate)

# 8) Definir a loss da SVM (hinge + regularização)
def svm_loss(X, y, w, b, C=1.0):
    # X: [N, d], y: [N], w: [d, 1], b: [1]
    scores = X @ w + b          # [N, 1]
    y = y.view(-1, 1)           # [N, 1]
    # hinge: max(0, 1 - y * score)
    hinge = torch.clamp(1 - y * scores, min=0)
    classification_loss = hinge.mean()
    reg_loss = 0.5 * torch.sum(w ** 2)
    total = reg_loss + C * classification_loss
    return total, classification_loss, reg_loss, scores

# 9) Treino
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    total_loss, cls_loss, reg_loss, scores = svm_loss(X_train_t, y_train_t, w, b, C=C)
    total_loss.backward()
    optimizer.step()

    if epoch % 50 == 0 or epoch == 1:
        # acurácia de treino
        with torch.no_grad():
            preds_train = torch.sign(scores).view(-1)
            acc_train = (preds_train == y_train_t).float().mean().item()
        print(f"[{epoch:3d}/{epochs}] loss={total_loss.item():.4f}  cls={cls_loss.item():.4f}  reg={reg_loss.item():.4f}  acc_train={acc_train:.3f}")

# 10) Avaliação no TESTE
with torch.no_grad():
    test_scores = X_test_t @ w + b
    preds_test = torch.sign(test_scores).view(-1)

acc_test = (preds_test == y_test_t).float().mean().item()
print("\n=== Avaliação no TESTE ===")
print(f"Acurácia: {acc_test:.3f}")

# matriz de confusão (convertendo p/ {0,1} só pra ficar bonita)
y_test_bin = (y_test_t == 1).int().numpy()
preds_test_bin = (preds_test == 1).int().numpy()
cm = confusion_matrix(y_test_bin, preds_test_bin)
print("Matriz de confusão:\n", cm)


## Execução do treinamento

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = X @ w + b  # Modelo SVM (um hiperplano que depende de w e b)

    # Hinge loss: max(0, 1 - y_i * (w^T x_i + b))
    perda_de_classificacao = torch.clamp(1 - y.view(-1, 1) * y_pred, min=0).mean()

    # Termo de regularização
    perda_de_distancia_entre_classes = 0.5 * torch.sum(w ** 2) # 2/||w|| é a distância que queremos que seja a maior possível

    # Função objetivo tradicional: minimizar reg + C * hinge
    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")